In [4]:
%matplotlib inline
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [16]:
data = pd.read_csv('data/labeledTrainData.tsv', sep='\t')
X = data.review.as_matrix()
y = data.sentiment.as_matrix()
del data['id']
data.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=4000, random_state=0, stratify=y)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_train_bag = vectorizer.fit_transform(X_train)
X_test_bag = vectorizer.transform(X_test)

In [14]:
most_frequent = argsort(sum(X_train_bag.A, axis=0))[::-1][:15000]
X_train_bag_frequent = X_train_bag[:, most_frequent]
X_test_bag_frequent = X_test_bag[:, most_frequent]

Постройте RF, Logistic Regression, SVM, KNN

Постройте *precision-recall* и *roc* кривые обученных выше классификаторов (каждый тип кривой на отдельном графике), а также вычислите площадь под ними. Для этого удобно воспользоваться функциями [precision_recall_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html), [roc_curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) и [auc](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html#sklearn.metrics.auc). Не забудьте, что в данном случае необходимо использовать метод *predict_proba* для получения оценки принадлежности к первому классу.

Разбейте данные на 5 фолдов и подберите параметры у лучшего из алгоритмов. Улучшились ли AUC ROC, AUC PRC?